In [139]:
import pickle
import pandas as pd
import glob2
import plotly.express as px
import plotly.graph_objects as go
from influxdb import DataFrameClient
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
from influxdb import InfluxDBClient
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import time
import pickle
from scipy import optimize

# Cluster studies

In [2]:
HV_pd=pd.read_pickle("HV_and_cluster_data.pickle")

In [8]:
def plot_parameter_by_gain(cluster_pd, param, layer):
    fig=px.scatter(x=cluster_pd[f"L{layer}_gain"],y=cluster_pd[param],color=cluster_pd.run.astype('str'),color_discrete_sequence=px.colors.qualitative.Light24)
    fig.update_xaxes(title="Gain [V]")
    if "charge" in param:
        title= param + " [fC] (avg 2h)"
    else:
        title= param + " (avg 2h)"
    fig.update_layout(title=dict(text=title + " Vs humidity", x=0.5))
    fig.update_yaxes(title=title)
    fig.show()

In [5]:
for key in HV_pd:
    print (key)

D_L1_IMonDet_Dr
D_L1_IMonDet_G1
D_L1_IMonDet_G2
D_L1_IMonDet_G3
D_L1_IMonDet_In
D_L1_IMonDet_T1
D_L1_IMonDet_T2
L1_IMonDet_Dr
L1_IMonDet_G1
L1_IMonDet_G2
L1_IMonDet_G3
L1_IMonDet_In
L1_IMonDet_T1
L1_IMonDet_T2
L1_V0Set_Dr
L1_V0Set_G1
L1_V0Set_G2
L1_V0Set_G3
L1_V0Set_T1
L1_V0Set_T2
L1_V0Set_In
D_L1_IMonReal_Dr
D_L1_IMonReal_G1
D_L1_IMonReal_G2
D_L1_IMonReal_G3
D_L1_IMonReal_In
D_L1_IMonReal_T1
D_L1_IMonReal_T2
L1_IMonReal_Dr
L1_IMonReal_G1
L1_IMonReal_G2
L1_IMonReal_G3
L1_IMonReal_In
L1_IMonReal_T1
L1_IMonReal_T2
L1_VMon_Dr
L1_VMon_G1
L1_VMon_G2
L1_VMon_G3
L1_VMon_T1
L1_VMon_T2
L1_VMon_In
D_L2_IMonDet_Dr
D_L2_IMonDet_G1
D_L2_IMonDet_G2
D_L2_IMonDet_G3
D_L2_IMonDet_In
D_L2_IMonDet_T1
D_L2_IMonDet_T2
L2_IMonDet_Dr
L2_IMonDet_G1
L2_IMonDet_G2
L2_IMonDet_G3
L2_IMonDet_In
L2_IMonDet_T1
L2_IMonDet_T2
L2_V0Set_Dr
L2_V0Set_G1
L2_V0Set_G2
L2_V0Set_G3
L2_V0Set_T1
L2_V0Set_T2
L2_V0Set_In
D_L2_IMonReal_Dr
D_L2_IMonReal_G1
D_L2_IMonReal_G2
D_L2_IMonReal_G3
D_L2_IMonReal_In
D_L2_IMonReal_T1
D_L2_IMon

In [81]:
HV_pd_sliced=HV_pd[HV_pd.L2_VMon_Dr>749]
HV_pd_aver_run=HV_pd_sliced.groupby("run", as_index=False).mean()
plot_parameter_by_gain(HV_pd_aver_run,"Cluster_charge_L2_X",2)

In [91]:

layer=1
HV_pd_sliced=HV_pd[HV_pd.L1_VMon_Dr>749][HV_pd.L1_gain>830][HV_pd.L1_gain<840]
HV_pd_aver_run=HV_pd_sliced.groupby("run", as_index=False).mean()
param="Cluster_charge_L1_X"
fig=px.scatter(x=HV_pd_aver_run.run,y=HV_pd_aver_run[param],color=HV_pd_aver_run.run.astype('str'),color_discrete_sequence=px.colors.qualitative.Light24)
fig.update_xaxes(title="Gain [V]")
if "charge" in param:
    title= param + " [fC] (avg 2h)"
else:
    title= param + " (avg 2h)"
fig.update_layout(title=dict(text=title + " Vs humidity", x=0.5))
fig.update_yaxes(title=title)
fig.show()

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [76]:
HV_pd_sliced=HV_pd[HV_pd.L2_VMon_Dr>740][HV_pd.run==418]

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [77]:
fig=make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=HV_pd_sliced.index, y=HV_pd_sliced.L2_V0Set_G1,mode="markers"))
fig.add_trace(go.Scatter(x=HV_pd_sliced.index, y=HV_pd_sliced.L2_V0Set_G2,mode="markers"))
fig.add_trace(go.Scatter(x=HV_pd_sliced.index, y=HV_pd_sliced.L2_V0Set_G3,mode="markers"))
# fig.add_trace(go.Scatter(x=HV_pd_sliced.index, y=HV_pd_sliced.subrun,mode="markers"))
# fig.add_trace(go.Scatter(x=HV_pd_sliced.index, y=HV_pd_sliced.subrun,mode="markers"))

In [302]:
HV_pd_sliced=HV_pd[HV_pd.L2_VMon_Dr>740][HV_pd.L1_VMon_Dr>740][HV_pd.run>372][HV_pd.run<379][HV_pd.Cluster_charge_L1_X>40]

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [303]:
px.box(x = ((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)).round(0),y=HV_pd_sliced.Cluster_charge_L1_X)

In [304]:
px.box(x = ((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)).round(0),y=HV_pd_sliced.Cluster_charge_L2_X)

In [305]:
px.box(x = ((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)).round(0),y=HV_pd_sliced.Cluster_charge_L1_V)

In [306]:
px.box(x = ((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)).round(0),y=HV_pd_sliced.Cluster_charge_L2_V)

In [307]:
def func(x, a,b ):
    return a * np.exp(b * x)

In [327]:
popt,pocv=optimize.curve_fit(func,((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),HV_pd_sliced.Cluster_charge_L2_V,p0=[0.00005,0.045])
fig=make_subplots()
fig.add_trace(
    go.Scatter(x = ((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),y=HV_pd_sliced.Cluster_charge_L2_V, mode= "markers"))
fig.add_trace(
    go.Scatter(x=np.arange(268,282), y=func(np.arange(268,282),*popt)))
print (popt)
fig.show()

[0.02921852 0.02898356]


In [313]:
# px.scatter(x = ((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),y=HV_pd_sliced.Cluster_charge_L2_V, color= HV_pd_sliced.run)

In [284]:
fig=make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=HV_pd.resample("120T").mean().index,y=HV_pd.resample("120T").mean().pressure))
fig.add_trace(
    go.Scatter(x=HV_pd.resample("120T").mean().index,y=HV_pd.resample("120T").mean().run), secondary_y=True)

In [329]:
popt1,pocv=optimize.curve_fit(func,((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),HV_pd_sliced.Cluster_charge_L1_V,p0=[0.00005,0.045])
popt2,pocv=optimize.curve_fit(func,((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),HV_pd_sliced.Cluster_charge_L2_V,p0=[0.00005,0.045])
popt3,pocv=optimize.curve_fit(func,((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),HV_pd_sliced.Cluster_charge_L1_X,p0=[0.00005,0.045])
popt4,pocv=optimize.curve_fit(func,((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),HV_pd_sliced.Cluster_charge_L2_X,p0=[0.00005,0.045])
popt=[popt1,popt2,popt3,popt4]

In [336]:
popt_avg= (np.average(popt,0))

In [346]:
HV_pd_sliced=HV_pd[HV_pd.L2_VMon_Dr>749]
HV_pd_aver_run=HV_pd_sliced.groupby("run", as_index=False).mean()
px.scatter(x=HV_pd_aver_run.L1_gain,y=HV_pd_aver_run.Cluster_charge_L1_X,color=HV_pd_aver_run.run)

In [356]:
HV_pd_sliced=HV_pd[HV_pd.L2_VMon_Dr>749][HV_pd.run==378]
HV_pd_aver_run=HV_pd_sliced.groupby("run", as_index=False).mean()
ipsilon=HV_pd_sliced.Cluster_charge_L1_X/(func(((HV_pd_sliced.temperature+253)/(HV_pd_sliced.pressure/1013)),popt_avg[0] ,popt_avg[1] ))
px.scatter(x=HV_pd_sliced.index,y=ipsilon/np.max(ipsilon),color=HV_pd_sliced.run)

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [353]:
px.scatter(,color=HV_pd_sliced.run)

In [369]:
fig = make_subplots()
fig.add_trace(
go.Scatter(x=HV_pd_sliced.index,y=ipsilon/np.max(ipsilon),mode="markers"))
fig.add_trace(
go.Scatter(x=HV_pd_sliced.index,y=HV_pd_sliced.Cluster_charge_L1_X/np.max(HV_pd_sliced.Cluster_charge_L1_X),mode="markers"))